# YELP

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName("yelp").getOrCreate()


load data

In [39]:
business_ds = spark.read.json("./data/yelp_academic_dataset_business.json")

business_ds.printSchema(1)
business_ds.show(5)

print("Розмірність датасету:")
print("К-сть колонок:", len(business_ds.columns))
print("К-сть записів:", business_ds.count())

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- hours: struct (nullable = true)
 |-- is_open: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------

In [40]:
review_ds = spark.read.json("./data/yelp_academic_dataset_review.json")

review_ds.printSchema(1)
review_ds.show(5)

print("Розмірність датасету:")
print("К-сть колонок:", len(review_ds.columns))
print("К-сть записів:", review_ds.count())

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPp

In [41]:
tip_ds = spark.read.json("./data/yelp_academic_dataset_tip.json")

tip_ds.printSchema(1)
tip_ds.show(5)

print("Розмірність датасету:")
print("К-сть колонок:", len(tip_ds.columns))
print("К-сть записів:", tip_ds.count())

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|3uLgwr0qeCNMjKenH...|               0|2012-05-18 02:17:21|Avengers time wit...|AGNUgVwnZUey3gcPC...|
|QoezRbYQncpRqyrLH...|               0|2013-02-05 18:35:10|They have lots of...|NBN4MgHP9D3cw--Sn...|
|MYoRNLb5chwjQe3c_...|               0|2013-08-18 00:56:08|It's open even wh...|-copOvldyKh1qr-vz...|
|hV-bABTK-glh5wj31...|               0|2017-06-27 23:05:38|Very decent fried...|FjMQVZjSqY8syIO-5...|
|_uN0OudeJ3Zl_tf6n...|               0|2012-10-06 19:43:09|Appetizers.. plat...|

In [42]:
user_ds = spark.read.json("./data/yelp_academic_dataset_user.json")

user_ds.printSchema(1)
user_ds.show(7)

print("Розмірність датасету:")
print("К-сть колонок:", len(user_ds.columns))
print("К-сть записів:", user_ds.count())

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

+-------------+---------------+---------------+----------------+-----

statistics about numerical columns

In [7]:
business_stats = business_ds.describe(["stars", "review_count"])
business_stats.show()

+-------+------------------+------------------+
|summary|             stars|      review_count|
+-------+------------------+------------------+
|  count|            150346|            150346|
|   mean|3.5967235576603303| 44.86656113232144|
| stddev|0.9744207509201349|121.12013570117087|
|    min|               1.0|                 5|
|    max|               5.0|              7568|
+-------+------------------+------------------+


In [8]:
review_stats = review_ds.describe(["funny", "stars", "useful"])
review_stats.show()

+-------+-------------------+------------------+------------------+
|summary|              funny|             stars|            useful|
+-------+-------------------+------------------+------------------+
|  count|            6990280|           6990280|           6990280|
|   mean|0.32655959417934616|  3.74858374771826|1.1846089140921394|
| stddev| 1.6887290985540495|1.4787045052556855| 3.253766966933363|
|    min|                 -1|               1.0|                -1|
|    max|                792|               5.0|              1182|
+-------+-------------------+------------------+------------------+


In [9]:
tip_stats = tip_ds.describe(["compliment_count"])
tip_stats.show()

+-------+--------------------+
|summary|    compliment_count|
+-------+--------------------+
|  count|              908915|
|   mean|0.012524823553357574|
| stddev| 0.12076339327984317|
|    min|                   0|
|    max|                   6|
+-------+--------------------+


In [10]:
user_stats = user_ds.describe(["average_stars", "cool", "fans", "funny", "review_count", "useful"])
user_stats.show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|     average_stars|              cool|              fans|             funny|      review_count|           useful|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|           1987897|           1987897|           1987897|           1987897|           1987897|          1987897|
|   mean|  3.63049415035087|23.792913817969442|1.4657404282012598|16.970536199813168|23.394409267683386|42.29633527290398|
| stddev|1.1833369995975145| 565.3512954713351| 18.13075272385579| 407.8034374841018| 82.56699161797889|641.4805967755902|
|    min|               1.0|                 0|                 0|                 0|                 0|                0|
|    max|               5.0|            199878|             12497|            185823|             17473|           206296|
+-------+-------

business questions

Як змінюється середній рейтинг бізнесів за відгуками у кожному місті протягом року?

In [36]:
reviews_with_month = review_ds.withColumn("month", f.month("date"))

windowSpec = Window.partitionBy("city", "month")

average_monthly_rating = (business_ds
    .join(reviews_with_month, business_ds.business_id == reviews_with_month.business_id)
    .withColumn("avg_monthly_rating", f.avg(reviews_with_month.stars).over(windowSpec))
    .select("city", "month", "avg_monthly_rating")
    .dropDuplicates(["city", "month"])
)


In [37]:
average_monthly_rating.show()

+------------+-----+------------------+
|        city|month|avg_monthly_rating|
+------------+-----+------------------+
| AB Edmonton|    9|               5.0|
|      AMBLER|   11|               3.0|
|      AMBLER|   12|3.6666666666666665|
|        AVON|    7|              3.25|
|    Abington|    3|  3.40997830802603|
|       Afton|    7|               4.4|
|       Afton|    8|              3.75|
|       Afton|   11|               3.5|
|     Algiers|    5|               1.0|
|     Algiers|    9|               1.0|
| Aliso Viejo|    9|               5.0|
|     Alloway|    4|               5.0|
|   Andalusia|    2|               5.0|
|     Antioch|    5|3.5048076923076925|
|Apollo Beach|    1|3.7840616966580978|
|Apollo Beach|    5| 3.596091205211726|
|Apollo Beach|    9|3.7129337539432177|
|Apollo beach|    7|               5.0|
|       Arabi|    8| 4.301369863013699|
|       Arabi|   12| 4.448275862068965|
+------------+-----+------------------+


In [38]:
average_monthly_rating.write.mode("overwrite").csv("./data/csv/average_monthly_rating", header=True)

Який середній рейтинг бізнесів в кожному місті, при умові що бізнеси які впливають на рейтинг мають більше ніж 100 відгуків?

In [14]:
average_rating_per_city = (business_ds
    .filter(f.col("review_count") > 100)
    .groupBy("city").agg(f.avg("stars"))
)

In [15]:
average_rating_per_city.show()

+------------------+------------------+
|              city|        avg(stars)|
+------------------+------------------+
|       Springfield|3.2954545454545454|
|   King Of Prussia|3.2045454545454546|
|Indian Rocks Beach|              3.86|
|    Lafayette Hill| 4.071428571428571|
|       Cherry Hill|3.6193181818181817|
|         Levittown|3.9444444444444446|
|        Brookhaven|               3.5|
|        Isla Vista|             3.825|
|     Virginia City|              4.25|
|         Blue Bell|             3.525|
|      West Chester|3.7083333333333335|
|          Gallatin| 4.666666666666667|
|     Saint Charles| 3.735294117647059|
|           Hatboro|            3.8125|
|         Hockessin|              4.25|
|         Glassboro|               3.5|
|          Boothwyn|               4.5|
|          Perkasie| 4.333333333333333|
|          Franklin|             3.716|
|      Mount Laurel|               3.4|
+------------------+------------------+


In [16]:
average_rating_per_city.write.mode("overwrite").csv(".\\data\\csv\\average_rating_per_city", header=True)

В якому році було залишено найбільше відгуків?

In [17]:
reviews_by_year = (review_ds
    .withColumn("year", f.year("date"))
    .groupBy("year").agg(f.count("review_id"))
    .orderBy(f.desc("count(review_id)"))
)


In [18]:
reviews_by_year.show()

+----+----------------+
|year|count(review_id)|
+----+----------------+
|2019|          907284|
|2018|          906362|
|2017|          820048|
|2016|          758882|
|2015|          688415|
|2021|          618189|
|2020|          554557|
|2014|          522275|
|2013|          383950|
|2012|          286570|
|2011|          230813|
|2010|          138587|
|2009|           74387|
|2008|           48226|
|2022|           31665|
|2007|           15363|
|2006|            3853|
|2005|             854|
+----+----------------+


In [19]:
reviews_by_year.write.mode("overwrite").csv(".\\data\\csv\\reviews_by_year", header=True)

Топ 10 бізнесів які мають найбільшу кількість оцінок в 1 зірку

In [20]:
top_10_businesses_one_star = (review_ds
    .filter(f.col("stars") == 1)
    .join(business_ds, review_ds.business_id == business_ds.business_id)
    .groupBy(review_ds["business_id"], "name", "categories")
    .count()
    .orderBy(f.desc("count"))
    .limit(10)
    .select("name", "categories", "count")
)

In [21]:
top_10_businesses_one_star.show(truncate=False)

+----------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|name                                    |categories                                                                                                                                                            |count|
+----------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Grand Sierra Resort and Casino          |Nightlife, Hotels & Travel, Arts & Entertainment, Music Venues, Hotels, Casinos, Event Planning & Services, Resorts                                                   |1045 |
|Geno's Steaks                           |Sandwiches, Cheesesteaks, Steakhouses, Restaurants                                            

In [22]:
top_10_businesses_one_star.write.mode("overwrite").csv(".\\data\\csv\\top_10_businesses_one_star", header=True)

Які категорії бізнесів мають найбільшу кількість п'ятизіркових відгуків?

In [23]:
business_wsc_ds = (business_ds
    .withColumn("categories", f.split(f.col("categories"), ", "))
    .withColumn("category", f.explode(f.col("categories")))
)

top_categories = (review_ds
    .filter(f.col("stars") == 5)
    .join(business_wsc_ds, "business_id")
    .groupBy("category")
    .count()
    .orderBy(f.desc("count"))
)

In [24]:
top_categories.show()

+--------------------+-------+
|            category|  count|
+--------------------+-------+
|         Restaurants|2079441|
|                Food| 868282|
|           Nightlife| 654140|
|                Bars| 617201|
|      American (New)| 432315|
|  Breakfast & Brunch| 400199|
|American (Traditi...| 399045|
|          Sandwiches| 317516|
|             Seafood| 281802|
|Event Planning & ...| 263540|
|            Shopping| 259557|
|       Beauty & Spas| 221478|
|        Coffee & Tea| 217011|
|               Pizza| 203007|
|             Italian| 195508|
|             Mexican| 188450|
|       Cocktail Bars| 163718|
|             Burgers| 163618|
|Arts & Entertainment| 157111|
|            Desserts| 156471|
+--------------------+-------+


In [25]:
top_categories.write.mode("overwrite").csv(".\\data\\csv\\top_categories", header=True)

Яка середня кількість відгуків на бізнес в кожному місті?

In [26]:
windowSpec = Window.partitionBy("city")

average_reviews_per_business = (business_ds
    .withColumn("avg_reviews", f.avg("review_count").over(windowSpec))
    .select("city", "avg_reviews")
    .dropDuplicates(["city"])
)

In [27]:
average_reviews_per_business.show(truncate=False)

+-----------------------+------------------+
|city                   |avg_reviews       |
+-----------------------+------------------+
|AB Edmonton            |6.0               |
|AMBLER                 |9.0               |
|ARDMORE                |8.0               |
|AVON                   |24.0              |
|Abington               |28.57396449704142 |
|Abington Township      |83.66666666666667 |
|Affton                 |25.27777777777778 |
|Afton                  |50.0              |
|Alberta Park Industrial|5.0               |
|Aldan                  |17.545454545454547|
|Algiers                |28.0              |
|Aliso Viejo            |9.0               |
|Alloway                |9.0               |
|Almonesson             |16.0              |
|Alton                  |23.053333333333335|
|Ambler                 |36.96208530805687 |
|Andalusia              |16.333333333333332|
|Antioch                |30.93073593073593 |
|Apollo Beach           |31.65573770491803 |
|Apollo be

In [28]:
average_reviews_per_business.write.mode("overwrite").csv(".\\data\\csv\\average_reviews_per_business", header=True)